# Job Scraper

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import pandas as pd
import re

In [2]:
# https://es.indeed.com/ofertas?q=data+scientist&l=Espa%C3%B1a
# URL encoding: España = Espa%C3%B1a
indeed = 'https://es.indeed.com/jobs?q={}&l={}&limit={}&start={}'

In [3]:
def get_url(job, location, limit, start):
    job = job.replace(' ', '+')
    url = indeed.format(job, location, limit, start)
    return url

In [4]:
# Get url with desired variables
url = get_url('data scientist', 'España', 50, 0)
url

'https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=0'

In [5]:
# Url check
requests.get(url)

<Response [200]>

In [6]:
def get_html(url):
    response = requests.get(url, timeout = 30)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return False

In [7]:
# Get html
soup = get_html(url)
soup

<!DOCTYPE html>

<html dir="ltr" lang="es">
<head>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<script src="//d3fw5vlhllyvee.cloudfront.net/s/732ac0d/es_ES.js" type="text/javascript"></script>
<script>window.indeed=window.indeed||{},indeed.crashtext=indeed.crashtext||{},indeed.crashtext.populate=indeed.crashtext.populate||function(e,n){function t(e){for(var n=a,t=e.length;t;)n=33*n^e.charCodeAt(--t);return n>>>0}var r=window[n];if(r){var i=r[""]||{},a=i.salt;if(i.hasOwnProperty("salt"))for(var d in e)r[function(e){var n=t(e);return i.hasOwnProperty("id_length")&&(n=String(n).substring(0,i.id_length)),n}(d)]=e[d];else for(var d in e)r[d]=[null].concat(e[d])}};indeed.crashtext.populate({"serp_title_q\u0004{0,choice,0#|1#|1<{0,number,integer} }{1} Jobs, Employment {2}| {3}":["{1}. Ofertas de trabajo en {3} España"],"serp_title_ql\u0004{0,choice,0#|1#|1<{0,number,integer} }{1} Jobs, Employment in {2} {3}| {4}":["{1} en {2}. Ofertas de trabajo en {4} España"],"serp_t

In [8]:
jobs = soup.find_all('div', 'jobsearch-SerpJobCard')
jobs

[<div class="jobsearch-SerpJobCard unifiedRow row result" data-jk="547cd179f403ae06" data-tn-component="organicJob" id="p_547cd179f403ae06">
 <h2 class="title">
 <a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/rc/clk?jk=547cd179f403ae06&amp;fccid=0e374b82a29a1049&amp;vjs=3" id="jl_547cd179f403ae06" onclick="setRefineByCookie([]); return rclk(this,jobmap[0],true,0);" onmousedown="return rclk(this,jobmap[0],0);" rel="noopener nofollow" target="_blank" title="Applied Data Scientist">
 Applied <b>Data</b> <b>Scientist</b></a>
 </h2>
 <div class="sjcl">
 <div>
 <span class="company">
 IntelliSense.io</span>
 </div>
 <div class="recJobLoc" data-rc-loc="Barcelona, Barcelona provincia" id="recJobLoc_547cd179f403ae06" style="display: none"></div>
 <span class="location accessible-contrast-color-location">Barcelona, Barcelona provincia</span>
 <span class="remote-bullet">•</span>
 <span class="remote">Teletrabajo</span>
 </div>
 <div class="summary">
 <ul style="list-style-ty

In [9]:
job0 = jobs[0]
job0

<div class="jobsearch-SerpJobCard unifiedRow row result" data-jk="547cd179f403ae06" data-tn-component="organicJob" id="p_547cd179f403ae06">
<h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/rc/clk?jk=547cd179f403ae06&amp;fccid=0e374b82a29a1049&amp;vjs=3" id="jl_547cd179f403ae06" onclick="setRefineByCookie([]); return rclk(this,jobmap[0],true,0);" onmousedown="return rclk(this,jobmap[0],0);" rel="noopener nofollow" target="_blank" title="Applied Data Scientist">
Applied <b>Data</b> <b>Scientist</b></a>
</h2>
<div class="sjcl">
<div>
<span class="company">
IntelliSense.io</span>
</div>
<div class="recJobLoc" data-rc-loc="Barcelona, Barcelona provincia" id="recJobLoc_547cd179f403ae06" style="display: none"></div>
<span class="location accessible-contrast-color-location">Barcelona, Barcelona provincia</span>
<span class="remote-bullet">•</span>
<span class="remote">Teletrabajo</span>
</div>
<div class="summary">
<ul style="list-style-type:circle;margin-

## Output example
  
From the list of jobs obtained we take a look in on of the jobs. As we can see most of the data we want to scrape are on the first half of the html response.

```html
<div class="jobsearch-SerpJobCard unifiedRow row result" data-jk="547cd179f403ae06" data-tn-component="organicJob" id="p_547cd179f403ae06">
<h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/rc/clk?jk=547cd179f403ae06&amp;fccid=0e374b82a29a1049&amp;vjs=3" id="jl_547cd179f403ae06" onclick="setRefineByCookie([]); return rclk(this,jobmap[0],true,0);" onmousedown="return rclk(this,jobmap[0],0);" rel="noopener nofollow" target="_blank" title="Applied Data Scientist">
Applied <b>Data</b> <b>Scientist</b></a>
<span class="new">nueva</span></h2>
<div class="sjcl">
<div>
<span class="company">
IntelliSense.io</span>
</div>
<div class="recJobLoc" data-rc-loc="Barcelona, Barcelona provincia" id="recJobLoc_547cd179f403ae06" style="display: none"></div>
<span class="location accessible-contrast-color-location">Barcelona, Barcelona provincia</span>
<span class="remote-bullet">•</span>
<span class="remote">Teletrabajo</span>
</div>
<div class="summary">
<ul style="list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;">
<li>The Applied <b>Data</b> <b>Scientist</b> will work on our world-class models including <b>data</b> assessment and preparation; updating and upgrading existing models, and developing…</li>
</ul></div>
<div class="jobsearch-SerpJobCard-footer">
<div class="jobsearch-SerpJobCard-footerActions">
<div class="result-link-bar-container">
<div class="result-link-bar"><span class="date date-a11y">hace 5 días</span><div class="tt_set" id="tt_set_0"><div class="job-reaction"><button aria-expanded="false" aria-haspopup="true" aria-label="guardar o desechar" class="job-reaction-kebab" data-ol-has-click-handler="" onclick="toggleKebabMenu('547cd179f403ae06', false, event); return false;" tabindex="0"></button><span class="job-reaction-kebab-menu"><button class="job-reaction-kebab-item job-reaction-save" data-ol-has-click-handler="" onclick="changeJobState('547cd179f403ae06', 'save', 'linkbar', false, '');return false;"><svg focusable="false" height="16" viewbox="0 0 24 24" width="16"><g><path d="M16.5,3A6,6,0,0,0,12,5.09,6,6,0,0,0,7.5,3,5.45,5.45,0,0,0,2,8.5C2,12.28,5.4,15.36,10.55,20L12,21.35,13.45,20C18.6,15.36,22,12.28,22,8.5A5.45,5.45,0,0,0,16.5,3ZM12.1,18.55l-0.1.1-0.1-.1C7.14,14.24,4,11.39,4,8.5A3.42,3.42,0,0,1,7.5,5a3.91,3.91,0,0,1,3.57,2.36h1.87A3.88,3.88,0,0,1,16.5,5,3.42,3.42,0,0,1,20,8.5C20,11.39,16.86,14.24,12.1,18.55Z" fill="#2d2d2d"></path></g></svg><span class="job-reaction-kebab-item-text">Guardar empleo</span></button><button class="job-reaction-kebab-item job-reaction-dislike" data-ol-has-click-handler="" onclick="dislikeJob(false, false, '547cd179f403ae06', 'unsave', 'linkbar', false, '');"><span class="job-reaction-dislike-icon"></span><span class="job-reaction-kebab-item-text">No me interesa</span></button><button class="job-reaction-kebab-item job-reaction-report" onclick="reportJob('547cd179f403ae06');"><span class="job-reaction-report-icon"></span><span class="job-reaction-kebab-item-text">Denunciar empleo</span></button></span></div><span class="result-link-bar-separator">·</span><a class="sl resultLink save-job-link" href="#" id="sj_547cd179f403ae06" onclick="changeJobState('547cd179f403ae06', 'save', 'linkbar', false, ''); return false;" title="Guardar esta oferta de empleo en mi.indeed">Guardar empleo</a><span class="result-link-bar-separator">·</span><button aria-expanded="false" class="sl resultLink more-link" id="tog_0" onclick="toggleMoreLinks('547cd179f403ae06', '0'); return false;">más...</button></div><script>if (!window['result_547cd179f403ae06']) {window['result_547cd179f403ae06'] = {};}window['result_547cd179f403ae06']['showSource'] = false; window['result_547cd179f403ae06']['source'] = "Mustakbil"; window['result_547cd179f403ae06']['loggedIn'] = false; window['result_547cd179f403ae06']['showMyJobsLinks'] = false;window['result_547cd179f403ae06']['baseMyJobsUrl'] = "https://myjobs.indeed.com";window['result_547cd179f403ae06']['undoAction'] = "unsave";window['result_547cd179f403ae06']['relativeJobAge'] = "hace 5 días";window['result_547cd179f403ae06']['jobKey'] = "547cd179f403ae06"; window['result_547cd179f403ae06']['myIndeedAvailable'] = true; window['result_547cd179f403ae06']['showMoreActionsLink'] = window['result_547cd179f403ae06']['showMoreActionsLink'] || true; window['result_547cd179f403ae06']['resultNumber'] = 0; window['result_547cd179f403ae06']['jobStateChangedToSaved'] = false; window['result_547cd179f403ae06']['searchState'] = "q=data scientist&amp;l=Espa%C3%B1a"; window['result_547cd179f403ae06']['basicPermaLink'] = "https://es.indeed.com"; window['result_547cd179f403ae06']['saveJobFailed'] = false; window['result_547cd179f403ae06']['removeJobFailed'] = false; window['result_547cd179f403ae06']['requestPending'] = false; window['result_547cd179f403ae06']['currentPage'] = "serp"; window['result_547cd179f403ae06']['sponsored'] = false;window['result_547cd179f403ae06']['reportJobButtonEnabled'] = false; window['result_547cd179f403ae06']['showMyJobsHired'] = false; window['result_547cd179f403ae06']['showSaveForSponsored'] = false; window['result_547cd179f403ae06']['showJobAge'] = true; window['result_547cd179f403ae06']['showHolisticCard'] = true; window['result_547cd179f403ae06']['showDislike'] = true; window['result_547cd179f403ae06']['showKebab'] = true; window['result_547cd179f403ae06']['showReport'] = true;</script></div></div>
</div>
</div>
<div class="tab-container">
<div class="more-links-container result-tab" id="tt_display_0" style="display:none;"><div class="more_actions" id="more_0"><ul><li><span class="mat">Ver todo: <a href="/Ofertas-de-Intellisense.io" rel="nofollow">Empleos de IntelliSense.io</a> - <a href="/Ofertas-en-Barcelona,-Barcelona-provincia">trabajo en Barcelona</a></span></li></ul></div><a class="close-link closeLink" href="#" onclick="toggleMoreLinks('547cd179f403ae06'); return false;" title="Close"></a></div><div class="dya-container result-tab"></div>
<div class="tellafriend-container result-tab email_job_content"></div>
<div class="sign-in-container result-tab"></div>
</div>
</div>
```

In [10]:
# Get job url
def get_job_url(job):
    job_url = 'https://es.indeed.com' + job.h2.a['href']
    return job_url

In [11]:
# Get job company and remove spaces
def get_company(job):
    try:
        return job.find('span', 'company').text.strip()
    except:
        return None

In [12]:
# Get job title
def get_title(job):
    return job.h2.a['title']

In [13]:
# Get job location
def get_job_loc(job):
    loc = job.find('div', 'recJobLoc')['data-rc-loc']
    if loc == ('En remoto' or 'Teletrabajo'):
        return None
    else:
        return loc

In [14]:
# Get remote
def get_remote(job):
    remote = job.find('span', 'remote')
    
    # loc = job.find('div', 'recJobLoc')['data-rc-loc']
    if remote != None or get_job_loc(job) == None:
        return True
    else:
        return False

In [15]:
# Get job post date reference
def get_post_date(job):
    return job.find('span', 'date date-a11y').text.strip()

In [16]:
# Get company rating
def get_rating(job):
    try:
        return job.find('span', 'ratingsContent').text.strip()
    except:
        return None

In [17]:
# Extraction date 
extract_date = datetime.today().strftime('%Y-%m-%d')
extract_date

'2021-03-29'

### Tests

In [18]:
get_job_url(jobs[2])

'https://es.indeed.com/rc/clk?jk=75db214fdeb5f6d1&fccid=34d04f6900db138d&vjs=3'

In [19]:
# Test
requests.get(get_job_url(jobs[2]))

<Response [200]>

In [20]:
print(get_job_loc(jobs[13]))

Barcelona, Barcelona provincia


In [21]:
get_remote(job0)

True

In [22]:
for job in jobs:
    print(get_title(job))
    print(get_company(job))
    print(get_rating(job))
    print(get_job_loc(job))
    print(get_remote(job))
    print(get_post_date(job))
    print(extract_date)
    print(get_job_url(job))
    print('--------------------------')

Applied Data Scientist
IntelliSense.io
None
Barcelona, Barcelona provincia
True
hace 8 días
2021-03-29
https://es.indeed.com/rc/clk?jk=547cd179f403ae06&fccid=0e374b82a29a1049&vjs=3
--------------------------
Data Scientist Intern
ZeptoLab
None
Barcelona, Barcelona provincia
False
hace 15 días
2021-03-29
https://es.indeed.com/rc/clk?jk=0a477c63722f41a7&fccid=fea8c93b7974f803&vjs=3
--------------------------
Data Scientist
eDreams ODIGEO
3,6
Barcelona, Barcelona provincia
True
hace 4 días
2021-03-29
https://es.indeed.com/rc/clk?jk=75db214fdeb5f6d1&fccid=34d04f6900db138d&vjs=3
--------------------------
Data Scientist
Jellyfish Group Ltd
None
Barcelona, Barcelona provincia
False
hace 2 días
2021-03-29
https://es.indeed.com/rc/clk?jk=5cc487de9c935075&fccid=00f1b8a7966fcb79&vjs=3
--------------------------
Data Scientist
Entravision
None
Barcelona, Barcelona provincia
True
hace 10 días
2021-03-29
https://es.indeed.com/rc/clk?jk=65c81475f16fb2ef&fccid=715e5538e133fb28&vjs=3
-----------------

## Creating pandas dataframe and inserting values

In [24]:
col_name = ['JobTitle', 'Company', 'CompanyRating', 'JobLocation', 'Remote', 'PostDate', 'ExtractionDate', 'JobUrl']
df2 = pd.DataFrame(columns=col_name)
for job in jobs:
    df2.loc[len(df2)] = [get_title(job), get_company(job), get_rating(job), get_job_loc(job), 
                       get_remote(job), get_post_date(job), extract_date, get_job_url(job)] 
df2

,JobTitle,Company,CompanyRating,JobLocation,Remote,PostDate,ExtractionDate,JobUrl
0,Applied Data Scientist,IntelliSense.io,None,"Barcelona, Barcelona provincia",True,hace 8 días,2021-03-29,https://es.indeed.com/rc/clk?jk=547cd179f403ae...
1,Data Scientist Intern,ZeptoLab,None,"Barcelona, Barcelona provincia",False,hace 15 días,2021-03-29,https://es.indeed.com/rc/clk?jk=0a477c63722f41...
2,Data Scientist,eDreams ODIGEO,"3,6","Barcelona, Barcelona provincia",True,hace 4 días,2021-03-29,https://es.indeed.com/rc/clk?jk=75db214fdeb5f6...
3,Data Scientist,Jellyfish Group Ltd,None,"Barcelona, Barcelona provincia",False,hace 2 días,2021-03-29,https://es.indeed.com/rc/clk?jk=5cc487de9c9350...
4,Data Scientist,Entravision,None,"Barcelona, Barcelona provincia",True,hace 10 días,2021-03-29,https://es.indeed.com/rc/clk?jk=65c81475f16fb2...
5,CRM Junior Data Scientist,Adam Foods,None,"Barcelona, Barcelona provincia",False,hace 13 días,2021-03-29,https://es.indeed.com/rc/clk?jk=8722b98c6736bc...
6,Data Scientist Intern - Full time,Entravision Communications Corporation,None,"Barcelona, Barcelona provincia",False,hace 6 días,2021-03-29,https://es.indeed.com/rc/clk?jk=8ae46c8c831b41...
7,Reference Data Scientist,Holaluz,"4,0","Barcelona, Barcelona provincia",False,hace 6 días,2021-03-29,https://es.indeed.com/rc/clk?jk=1c5f89107bd4b4...
8,Data Scientist,Enzyme,None,"Barcelona, Barcelona provincia",False,Hace más de 30 días,2021-03-29,https://es.indeed.com/rc/clk?jk=a7a64c3537fc09...
9,Data Scientist Internship - Full time,Entravision,None,"Barcelona, Barcelona provincia",False,hace 10 días,2021-03-29,https://es.indeed.com/rc/clk?jk=fb89f55e0fdb0a...


In [26]:
# Check data with Url
df2.iloc[10]['JobUrl']

'https://es.indeed.com/rc/clk?jk=532353f2a63dac4d&fccid=8ef7e881dac94697&vjs=3'

# Main

In [27]:
loop = 0
limit = 50
max_job = 8 * limit
col_name = ['JobTitle', 'Company', 'CompanyRating', 'JobLocation', 'Remote', 'PostDate', 'ExtractionDate', 'JobUrl']

df = pd.DataFrame(columns=col_name)

while loop <= max_job:
    url = get_url('data scientist', 'España', limit, loop)
    print(url)
    soup = get_html(url) 
    jobs = soup.find_all('div', 'jobsearch-SerpJobCard')
    
    for job in jobs:
        df.loc[len(df)] = [get_title(job), get_company(job), get_rating(job), get_job_loc(job), 
                           get_remote(job), get_post_date(job), extract_date, get_job_url(job)]
    
    print('Page ', int((loop/50)+1), ' done!')
    time.sleep(30)
    loop += limit
print('Job done!')

https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=0
Page  1  done!
https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=50
Page  2  done!
https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=100
Page  3  done!
https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=150
Page  4  done!
https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=200
Page  5  done!
https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=250
Page  6  done!
https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=300
Page  7  done!
https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=350
Page  8  done!
https://es.indeed.com/jobs?q=data+scientist&l=España&limit=50&start=400
Page  9  done!
Job done!


In [28]:
df

,JobTitle,Company,CompanyRating,JobLocation,Remote,PostDate,ExtractionDate,JobUrl
0,Applied Data Scientist,IntelliSense.io,None,"Barcelona, Barcelona provincia",True,hace 8 días,2021-03-29,https://es.indeed.com/rc/clk?jk=547cd179f403ae...
1,Data Scientist Intern,ZeptoLab,None,"Barcelona, Barcelona provincia",False,hace 15 días,2021-03-29,https://es.indeed.com/rc/clk?jk=0a477c63722f41...
2,CRM Junior Data Scientist,Adam Foods,None,"Barcelona, Barcelona provincia",False,hace 13 días,2021-03-29,https://es.indeed.com/rc/clk?jk=8722b98c6736bc...
3,Data Scientist,eDreams ODIGEO,"3,6","Barcelona, Barcelona provincia",True,hace 4 días,2021-03-29,https://es.indeed.com/rc/clk?jk=75db214fdeb5f6...
4,Data Scientist,Jellyfish Group Ltd,None,"Barcelona, Barcelona provincia",False,hace 2 días,2021-03-29,https://es.indeed.com/rc/clk?jk=5cc487de9c9350...
...,...,...,...,...,...,...,...,...
430,Senior Software Developer - Linux C++,Malwarebytes,None,"Madrid, Madrid provincia",False,hace 25 días,2021-03-29,https://es.indeed.com/rc/clk?jk=63f82a272b89ef...
431,Cloud DevOps,B2M,None,"Madrid, Madrid provincia",False,Hace más de 30 días,2021-03-29,https://es.indeed.com/rc/clk?jk=e697647eeb0a95...
432,Senior Product Manager (m/f/x) Celonis Studio ...,Celonis,None,"Madrid, Madrid provincia",False,Hace más de 30 días,2021-03-29,https://es.indeed.com/rc/clk?jk=d1c8cc0c1e6a18...
433,"Research Scientist, Predictive Sciences (Netwo...",Bristol Myers Squibb,"4,2","Sevilla, Sevilla provincia",False,Hace más de 30 días,2021-03-29,https://es.indeed.com/pagead/clk?mo=r&ad=-6NYl...


In [30]:

df.to_csv (r'C:\Users\fzhu\OneDrive\UOC\Semestre 3\Tipologia i cicle de vida de les dades\PAC i Pràctiques\Pràctica 1\Jobs.csv', index=None)